In [1]:
import pandas as pd

In [4]:
metadata = pd.read_csv('./data/movies_metadata.csv', low_memory=False)

In [6]:
metadata.head()

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview  ... release_date  \
0  Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2  A family wedding reignites the ancient feud be...  ...   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4  Just when George Banks has recovered from his ...  ...   1995-02-10   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   
3  Released  Friends are the people who let you be yourself...   
4  Released  Just When His World Is Back To Normal... He's ...   

                         title  video vote_average vote_count  
0                    Toy Story  False          7.7     5415.0  
1                      Jumanji  False          6.9     2413.0  
2             Grumpier Old Men  False          6.5       92.0  
3            Waiting to Exhale  False          6.1       34.0  
4  Father of the Bride Part II  False          5.7      173.0  

[5 rows x 24 columns]

## 1. Simple Recommender ("Tell me what's trending")

The simplest way is to rank the movies based on their global user ratings. 

However, we have to consider the popularity of the movie as well. Otherwise, a movie with a rating of 9 from 10 voters will be considered 'better' than a movie with a rating of 8.9 from 10,000 voters.

This is pretty much like how we choose a restaurant, will you choose a 5-stat restaurant rated by only 5 people or a restaurant with 4.5 ratings from 1000 people? 

Thus, we will use a weighted rating for our simple recommender here:


$$WeightedRating (WR) = \frac{V}{V + V_{min}} \cdot R + \frac{V_{min}}{V + V_{min}} \cdot R_{avg} $$

Here 
* $V$ is the number of votes for this movie
* $V_{min}$ is the minimum number of votes for a movie to be considered
* $R$ is the average rating for this movie
* $R_{avg}$ is the average rating for all movies in our dataset

In [7]:
R_avg = metadata['vote_average'].mean()

R_avg

5.618207215134185

In [8]:
# Calculate minimun # of votes for a movie to be considered

V_min = metadata['vote_count'].quantile(0.9)    # Choose first 90% movies

V_min

160.0

In [10]:
# Create dataset for those movies with >= 160 votes

movie_cp = metadata.copy().loc[metadata['vote_count'] >= V_min]

movie_cp.shape

(4555, 24)

In [14]:
def weighted_rating(df, V_min=V_min, R_avg=R_avg):
    V = df['vote_count']
    R = df['vote_average']

    # Calculation based on the IMDB formula
    return (V/(V+V_min) * R) + (V_min/(V_min+V) * R_avg)

In [15]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
movie_cp['weighted_score'] = movie_cp.apply(weighted_rating, axis=1)

In [17]:
#Sort movies based on score calculated above
movie_cp = movie_cp.sort_values('weighted_score', ascending=False)

#Print the top 15 movies
movie_cp[['title', 'vote_count', 'vote_average', 'weighted_score']].head(20)

title  vote_count  vote_average  \
314           The Shawshank Redemption      8358.0           8.5   
834                      The Godfather      6024.0           8.5   
10309      Dilwale Dulhania Le Jayenge       661.0           9.1   
12481                  The Dark Knight     12269.0           8.3   
2843                        Fight Club      9678.0           8.3   
292                       Pulp Fiction      8670.0           8.3   
522                   Schindler's List      4436.0           8.3   
23673                         Whiplash      4376.0           8.3   
5481                     Spirited Away      3968.0           8.3   
2211                 Life Is Beautiful      3643.0           8.3   
1178            The Godfather: Part II      3418.0           8.3   
1152   One Flew Over the Cuckoo's Nest      3001.0           8.3   
351                       Forrest Gump      8147.0           8.2   
1154           The Empire Strikes Back      5998.0           8.2   
1176                            Psycho      2405.0           8.3   
18465                 The Intouchables      5410.0           8.2   
40251                       Your Name.      1030.0           8.5   
289             Leon: The Professional      4293.0           8.2   
3030                    The Green Mile      4166.0           8.2   
1170                        GoodFellas      3211.0           8.2   

       weighted_score  
314          8.445869  
834          8.425439  
10309        8.421453  
12481        8.265477  
2843         8.256385  
292          8.251406  
522          8.206639  
23673        8.205404  
5481         8.196055  
2211         8.187171  
1178         8.180076  
1152         8.164256  
351          8.150272  
1154         8.132919  
1176         8.132715  
18465        8.125837  
40251        8.112532  
289          8.107234  
3030         8.104511  
1170         8.077459

## 2. Content Based Recommender

However, our simple recommender still have some problem: What if a fantastic movie just got released? Isn't it unfair to exclude it from our list?

Plus, our user might not want to blindly follow the public, so recommending based on global rating is a little bit limited.